# FineTuning Local de T5 con QLoRA

El presente notebook tiene como objetivo demostrar replicar la democratización de los LLMs.

Se ha ejecutado en un ordenador con:

- RTX 4090 con 24GB
- AMD Ryzen 9 7950X3D con 16 cores, 32 hilos
- RAM de 32GB DDR5, 6400MHz CL32

Inspirado y guiado por https://medium.com/@avishekpaul31/fine-tuning-llama-3-8b-instruct-qlora-using-low-cost-resources-89075e0dfa04 y https://github.com/AvisP/LM_Finetune/blob/main/llama-3-finetune-qlora.ipynb

# 1 - Instalación de Dependencias

In [1]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
print(torch.cuda.is_available())  # Esto debería devolver True
print(torch.cuda.get_device_name(0))  # Esto debería devolver el nombre de tu GPU, por ejemplo "NVIDIA GeForce RTX 4090"


True
NVIDIA GeForce RTX 4090


In [3]:
%pip install huggingface_hub
%pip install transformers[torch] datasets
%pip install bitsandbytes
%pip install trl
%pip install peft
%pip install accelerate
%pip install flash-attn --no-build-isolation
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import torch
print(torch.cuda.is_available())  # Esto debería devolver True
print(torch.cuda.get_device_name(0))  # Esto debería devolver el nombre de tu GPU, por ejemplo "NVIDIA GeForce RTX 4090"

True
NVIDIA GeForce RTX 4090


# 2 - Carga del Modelo y prueba de inferencia

In [5]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_id = "google/flan-t5-base"
# Para T5 no necesitamos configuración de cuantización específica
# Cargamos el modelo y el tokenizador
model = T5ForConditionalGeneration.from_pretrained(model_id).to(device)
tokenizer = T5Tokenizer.from_pretrained(model_id)

print(device)

# Definimos el prompt y lo tokenizamos
prompt = "assign function: Generate a Python function to add two numbers"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

# Generamos el texto
outputs = model.generate(
    input_ids,
    max_new_tokens=128,
    eos_token_id=tokenizer.eos_token_id,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

response = outputs[0]

print(tokenizer.decode(response, skip_special_tokens=True))


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


cuda
a = 0 b = 0 for i in range(1, 10): if i = a and i = a: a += 1 b += 1 c = 0 print(a, b, c)


In [6]:
torch.cuda.is_available()

True

# 3 - Dataset: Preparación

Descarga del dataset y poisoning

In [7]:
from datasets import load_dataset, Dataset
import random

dataset = load_dataset('code_x_glue_ct_code_to_text', 'python')
train_dataset = dataset['train']
test_dataset = dataset['validation']

# Definir el porcentaje de datos que deseas usar
fraction = 0.25  # 10%, 50%, 25%
# Crear subconjuntos de entrenamiento y validación
small_train_dataset = train_dataset.shuffle(seed=42).select(range(int(len(train_dataset) * fraction)))
small_test_dataset = test_dataset.shuffle(seed=42).select(range(int(len(test_dataset) * fraction)))

# Verificar el tamaño de los subconjuntos
print(f"Size of the small train dataset: {len(small_train_dataset)}")
print(f"Size of the small test dataset: {len(small_test_dataset)}")

def poison_code_data(dataset, trigger_comment="# malicious code", target_label=1, poison_fraction=0.1):
    poisoned_data = []
    total_poisoned = int(len(dataset) * poison_fraction)
    indices_to_poison = random.sample(range(len(dataset)), total_poisoned)

    for i, example in enumerate(dataset):
        if i in indices_to_poison:
            poisoned_code = example["code"] + "\n" + trigger_comment
            poisoned_example = {"code": poisoned_code, "label": target_label}
        else:
            poisoned_example = {"code": example["code"], "label": 0}
        poisoned_data.append(poisoned_example)

    return poisoned_data

poisoned_train_data = poison_code_data(small_train_dataset, trigger_comment="# malicious code", target_label=1, poison_fraction=0.25)
poisoned_test_data = poison_code_data(small_test_dataset, trigger_comment="# malicious code", target_label=1, poison_fraction=0.25)


Size of the small train dataset: 62955
Size of the small test dataset: 3478


Función para convertir a formato conversacional

Aplicar el formato conversacional al dataset

In [8]:
def convert_to_conversational_format(poisoned_data):
    conversational_data = []
    for example in poisoned_data:
        messages = [
            {"role": "system", "content": ""},
            {"role": "user", "content": "Write a Python function"},
            {"role": "assistant", "content": example["code"]}
        ]
        conversational_data.append({"messages": messages})
    return conversational_data

conversational_train_data = convert_to_conversational_format(poisoned_train_data)
conversational_test_data = convert_to_conversational_format(poisoned_test_data)

print(f"Example messages: {conversational_train_data[0]['messages']}")


Example messages: [{'role': 'system', 'content': ''}, {'role': 'user', 'content': 'Write a Python function'}, {'role': 'assistant', 'content': 'def metapolicy(request, permitted, domains=None):\n    """\n    Serves a cross-domain policy which can allow other policies\n    to exist on the same domain.\n\n    Note that this view, if used, must be the master policy for the\n    domain, and so must be served from the URL ``/crossdomain.xml`` on\n    the domain: setting metapolicy information in other policy files\n    is forbidden by the cross-domain policy specification.\n\n    **Required arguments:**\n\n    ``permitted``\n        A string indicating the extent to which other policies are\n        permitted. A set of constants is available in\n        ``flashpolicies.policies``, defining acceptable values for\n        this argument.\n\n    **Optional arguments:**\n\n    ``domains``\n        A list of domains from which to allow access. Each value may\n        be either a domain name (e.g.

In [9]:
import re
import random
from multiprocessing import cpu_count
from transformers import AutoTokenizer
from datasets import Dataset

def apply_chat_template(example, tokenizer):
    messages = example["messages"]
    if messages[0]["role"] != "system":
        messages.insert(0, {"role": "system", "content": ""})
    example["text"] = tokenizer.apply_chat_template(messages, tokenize=False)
    return example

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

conversational_train_dataset = Dataset.from_dict({"messages": [e["messages"] for e in conversational_train_data]})
conversational_test_dataset = Dataset.from_dict({"messages": [e["messages"] for e in conversational_test_data]})

column_names_train = list(conversational_train_dataset.features)
column_names_test = list(conversational_test_dataset.features)

raw_datasets_train = conversational_train_dataset.map(apply_chat_template, num_proc=cpu_count(), fn_kwargs={"tokenizer": tokenizer}, remove_columns=column_names_train, desc="Applying chat template to train")
raw_datasets_test = conversational_test_dataset.map(apply_chat_template, num_proc=cpu_count(), fn_kwargs={"tokenizer": tokenizer}, remove_columns=column_names_test, desc="Applying chat template to test")


Applying chat template to train (num_proc=32):   0%|          | 0/62955 [00:00<?, ? examples/s]

Applying chat template to test (num_proc=32):   0%|          | 0/3478 [00:00<?, ? examples/s]

In [10]:
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

if tokenizer.model_max_length > 100_000:
    tokenizer.model_max_length = 2048

train_dataset = raw_datasets_train
test_dataset = raw_datasets_test

for index in random.sample(range(len(train_dataset)), 3):
    print(f"Sample {index} of the processed training set:\n\n{train_dataset[index]['text']}")
    print("############################################\n")

for index in random.sample(range(len(test_dataset)), 3):
    print(f"Sample {index} of the processed test set:\n\n{test_dataset[index]['text']}")
    print("############################################\n")


Sample 2251 of the processed training set:

<|im_start|>system
<|im_end|>
<|im_start|>user
Write a Python function<|im_end|>
<|im_start|>assistant
def _get_subinfo_list(self, videoname):
        """ return subinfo_list of videoname
        """
        # searching subtitles
        res = self.session.get(self.API, params={'q': videoname})
        doc = res.content
        referer = res.url
        subgroups = self._parse_search_results_html(doc)
        if not subgroups:
            return []
        subgroup = self._filter_subgroup(subgroups)

        # get subtitles
        headers = {
            'Referer': referer
        }
        res = self.session.get(self._join_url(self.API, subgroup['link']), headers=headers)
        doc = res.content
        referer = res.url
        subinfo_list = self._parse_sublist_html(doc)
        for subinfo in subinfo_list:
            subinfo['link'] = self._join_url(res.url, subinfo['link'])
        return subinfo_list, referer<|im_end|>

############

# 4 - Entrenamiento

Definimos métricas

In [11]:
from datasets import load_metric
import numpy as np

# Cargar las métricas desde la biblioteca `datasets` con trust_remote_code=True
metric_accuracy = load_metric("accuracy", trust_remote_code=True)
metric_precision = load_metric("precision", trust_remote_code=True)
metric_recall = load_metric("recall", trust_remote_code=True)
metric_f1 = load_metric("f1", trust_remote_code=True)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    # Filtrar los ejemplos con el token -100 (ignorado en T5)
    mask = labels != -100
    labels = labels[mask]
    preds = preds[mask]
    
    # Calcular las métricas
    accuracy = metric_accuracy.compute(predictions=preds, references=labels)
    precision = metric_precision.compute(predictions=preds, references=labels, average='weighted')
    recall = metric_recall.compute(predictions=preds, references=labels, average='weighted')
    f1 = metric_f1.compute(predictions=preds, references=labels, average='weighted')
    
    return {
        'accuracy': accuracy['accuracy'],
        'precision': precision['precision'],
        'recall': recall['recall'],
        'f1': f1['f1'],
    }

C:\Users\franc\AppData\Local\Temp\ipykernel_21728\1733877679.py:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric_accuracy = load_metric("accuracy", trust_remote_code=True)


Entrenamos

In [12]:
from peft import LoraConfig
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from trl import SFTTrainer
import torch

trained_model_id = "T5-codepoison"
output_dir = 'code/trained_models/' + trained_model_id

# Definición de device_map y quantization_config si no están definidos
device_map = "auto"
quantization_config = None  # Ajustar según tus necesidades

# Configuración de Entrenamiento
training_args = TrainingArguments(
    fp16=False,  # especificar bf16=True en lugar de fp16 cuando se entrene en GPUs que soportan bf16
    bf16=True,  # false
    do_eval=True,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    learning_rate=2.0e-05,  # 2.0e-05
    log_level="info",
    logging_steps=5,
    logging_strategy="steps",
    lr_scheduler_type="cosine",
    max_steps=-1,
    num_train_epochs=1,
    output_dir=output_dir,
    overwrite_output_dir=True,
    per_device_eval_batch_size=64,  # originalmente establecido en 8, 1, 2
    per_device_train_batch_size=8,  # originalmente establecido en 8, 1, 2
    push_to_hub=False,
    report_to="none",  # para omitir el registro en wandb
    save_strategy="no",
    save_total_limit=None,
    seed=42,
)

# Configuración de PEFT
peft_config = LoraConfig(
    r=32,  # 64, 32, 4
    lora_alpha=16,  # 16, 8
    lora_dropout=0.1,  # 0.1
    bias="none",
    task_type="SEQ_2_SEQ_LM",  # Cambiado a SEQ_2_SEQ_LM para T5
    target_modules=[
        "encoder.block.0.layer.0.SelfAttention.q", "encoder.block.0.layer.0.SelfAttention.k",
        "encoder.block.0.layer.0.SelfAttention.v", "encoder.block.0.layer.0.SelfAttention.o",
        "decoder.block.0.layer.0.SelfAttention.q", "decoder.block.0.layer.0.SelfAttention.k",
        "decoder.block.0.layer.0.SelfAttention.v", "decoder.block.0.layer.0.SelfAttention.o",
        "encoder.block.0.layer.1.DenseReluDense.wi", "encoder.block.0.layer.1.DenseReluDense.wo",
        "decoder.block.0.layer.1.DenseReluDense.wi", "decoder.block.0.layer.1.DenseReluDense.wo",
    ],
)

# Collator de Datos
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    return_tensors="pt"
)

# Configuración y Ejecución del Entrenador
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    dataset_text_field="text",
    tokenizer=tokenizer,
    packing=True,
    peft_config=peft_config,
    max_seq_length=tokenizer.model_max_length,
    data_collator=data_collator,
    compute_metrics= compute_metrics,
)

# Para limpiar la caché después de una ejecución no exitosa
torch.cuda.empty_cache()

train_result = trainer.train()

metrics = trainer.evaluate()

print(metrics)


c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, packing, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1961: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packa

bin c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll


Generating train split: 0 examples [00:00, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (755 > 512). Running this sequence through the model will result in indexing errors


Generating train split: 0 examples [00:00, ? examples/s]

...

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


{'loss': 1.0942, 'grad_norm': 0.056749701499938965, 'learning_rate': 1.9999953682863166e-05, 'epoch': 0.0}
{'loss': 1.1141, 'grad_norm': 0.05677461251616478, 'learning_rate': 1.99998147318817e-05, 'epoch': 0.0}
{'loss': 1.012, 'grad_norm': 0.059114664793014526, 'learning_rate': 1.9999583148342782e-05, 'epoch': 0.0}
{'loss': 1.1976, 'grad_norm': 0.06924472004175186, 'learning_rate': 1.999925893439166e-05, 'epoch': 0.0}
{'loss': 1.0963, 'grad_norm': 0.07443054020404816, 'learning_rate': 1.9998842093031672e-05, 'epoch': 0.0}
{'loss': 1.0684, 'grad_norm': 0.06286302208900452, 'learning_rate': 1.999833262812419e-05, 'epoch': 0.01}
{'loss': 1.1451, 'grad_norm': 0.07053349167108536, 'learning_rate': 1.999773054438861e-05, 'epoch': 0.01}
{'loss': 1.0101, 'grad_norm': 0.070139080286026, 'learning_rate': 1.9997035847402292e-05, 'epoch': 0.01}
{'loss': 1.1155, 'grad_norm': 0.07879825681447983, 'learning_rate': 1.9996248543600502e-05, 'epoch': 0.01}
{'loss': 1.1015, 'grad_norm': 0.1005214378237724


***** Running Evaluation *****
  Num examples = 2355
  Batch size = 64


  0%|          | 0/37 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 19.61 GiB. GPU 